<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #6: Evaluate machine learning models</h1>
<h3 align="right">Written by the KSO Team</h3>

# 1. Set up and requirements

### Install and import Python packages

In [ ]:
from IPython.display import clear_output

try:
    import google.colab
    import os

    IN_COLAB = True
    print("Running in Colab...")

    # Clone repo
    !git clone --recurse-submodules https://github.com/ocean-data-factory-sweden/koster_yolov4.git
    %pip install -q --upgrade pip
    %pip install -qr <(sed '/Pillow/d;/ipywidgets/d' koster_yolov4/requirements.txt)
    %pip install -qr <(sed '/Pillow/d;/ipywidgets/d' koster_yolov4/yolov5_tracker/requirements.txt)
    %pip install -qr <(sed '/Pillow/d;/ipywidgets/d' koster_yolov4/kso_utils/requirements.txt)

    # Fix libmagic issue
    !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

    # Replace upsampling script with custom version
    os.chdir("koster_yolov4/tutorials")
    sys.path.append("..")
    !mv ../src/upsampling.py /usr/local/lib/python3.7/dist-packages/torch/nn/modules/upsampling.py

    # Replace nearest neighbours script with custom version (due to relative path issue)
    !cp ../src/multi_tracker_zoo.py ../yolov5_tracker/trackers/strong_sort/multi_tracker_zoo.py

    # Enable external widgets
    from google.colab import output

    output.enable_custom_widget_manager()

    # Ensure widgets are shown properly
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension enable --user --py jupyter_bbox_widget

    print("All packages are installed and ready to go!")
    try:
        clear_output()
        print("All packages are installed and ready to go!")
    except:
        clear_output()
        print("There have been some issues installing the packages!")
except:
    IN_COLAB = False
    import pkgutil
    
    if (
        pkgutil.find_loader("torch") is None
        or pkgutil.find_loader("torchvision") is None
    ):
        %pip install -q --upgrade pip
        %pip install -q torch==1.8.0 torchvision==0.9.0

    # Replace nearest neighbours script with custom version (due to relative path issue)
    !cp ../src/multi_tracker_zoo.py ../yolov5_tracker/trackers/strong_sort/multi_tracker_zoo.py
    # Ensure widgets are shown properly
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension enable --user --py jupyter_bbox_widget
    clear_output()
    print("Running locally... you're good to go!")

In [ ]:
# Set the directory of the libraries
import sys, os
from pathlib import Path

sys.path.append("..")

# Enables testing changes in utils
%load_ext autoreload
%autoreload 2

# Set to display dataframes as interactive tables
from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.project_utils as p_utils
import kso_utils.t6_utils as t6
from kso_utils.project import ProjectProcessor, MLProjectProcessor

clear_output()
print("Packages loaded successfully")

# 2. Evaluate model on custom footage

### Choose your project

In [ ]:
project_name = t_utils.choose_project()

In [ ]:
project = p_utils.find_project(project_name=project_name.value)

In [ ]:
# Initialise pp
pp = ProjectProcessor(project)

In [ ]:
# Initialise mlp
mlp = MLProjectProcessor(pp)

### Choose model

In [ ]:
model = t6.choose_model(project_name.value, team_name=mlp.team_name)

### Download model

In [ ]:
download_dir = t_utils.choose_folder(".", "where to download the model")

In [ ]:
artifact_dir = mlp.get_model(model.value, download_dir.selected)

### Choose custom footage

In [ ]:
source = t_utils.choose_footage(
    project,
    project.movie_folder if project.movie_folder not in [None, "None"] else ".",
    "custom footage",
)

In [ ]:
source_value = t_utils.process_source(source)

### Choose where to save runs (this should be left as default value in most cases)

In [ ]:
save_dir = t_utils.choose_folder(".", "runs output")

### Run model over selected custom footage

In [ ]:
conf_thres = t6.choose_conf()

In [ ]:
mlp.detect_yolov5(
    source=source_value,
    save_dir=save_dir.selected,
    conf_thres=conf_thres.value,
    artifact_dir=artifact_dir,
)

In [ ]:
eval_dir = t_utils.choose_folder(
    save_dir.selected
    if "save_dir" in vars() and save_dir.selected is not None
    else ".",
    "runs output",
)

In [ ]:
mlp.save_detections_wandb(conf_thres.value, model.value, eval_dir.selected)

### View model output

In [ ]:
viewer = t6.choose_files(eval_dir.selected)

### View training and validation datasets (Only image data)

In [ ]:
train_dataset, val_dataset = t6.get_dataset(project_name.value, model.value)

#### Training set

In [ ]:
t6.get_data_viewer(os.path.join(train_dataset, "data/images"))

#### Validation set

In [ ]:
t6.get_data_viewer(os.path.join(val_dataset, "data/images"))

# 3. Optional Step: Track unique individuals

In [ ]:
mlp.track_individuals(
    source=source_value,
    artifact_dir=artifact_dir,
    eval_dir=eval_dir.selected,
    conf_thres=conf_thres.value,
    img_size=(540, 540),
)

In [ ]:
# END